# OGC API Processes demonstratie client

De Pygeoapi server kun je starten met `docker compose up`

Deze zou dan beschikbaar moeten zijn op http://localhost:5000/


In [3]:
import folium
import requests  # For making HTTP requests
import geopandas as gpd
import json

In [ ]:
url = "http://localhost:5000/processes/"  
 
response = requests.get(url)

response.text

In [8]:
def handle_response(response):
  if response.status_code == 200:
    
    return response.json()
    
  else:
    print(f"Error: {response.status_code, response.json()['description']}")

In [ ]:
url = "http://localhost:5000/processes/"  

headers = {"Content-Type": "application/json"}

response = requests.get(url, headers=headers)
processes =  handle_response(response)

processes

Kaart met de waarneemstations (OGC API Features)

In [7]:
m = folium.Map(location=[52.2, 5.5], zoom_start=8)

url = "http://localhost:5000/collections/knmi_meetstations/items?f=json&limit=1000"

# Fetch the GeoJSON data
gdf = gpd.read_file(url)

#quick hack omdat de datetime kolom niet goed naar geojson geconverteerd wordt...
gdf_subset = gdf[['STN','TYPE', 'geometry']]

# Convert to GeoJSON
geojson = gdf_subset.to_json()

folium.GeoJson(
    gdf_subset,
    name="Stations",
    style_function=lambda x: {'fillColor': 'blue', 'color': 'black', 'weight': 1, 'fillOpacity': 0.5},
    tooltip=folium.GeoJsonTooltip(fields=["STN"]),
    popup=folium.GeoJsonPopup(fields=["STN", "TYPE"]),
    highlight_function=lambda x: {"fillOpacity": 0.8},
).add_to(m)

m

Kaart met het resultaat van de classificatie door het OGC API Processes localoutlier algoritme 

In [18]:
def calc_lof(feat):
    url = "http://localhost:5000/processes/localoutlier/execution"  
    data = {"inputs": {"dataset": feat}}
    headers = {"Content-Type": "application/json"}

    response = requests.post(url, json=data, headers=headers)
    return handle_response(response)


#let op: omdat dit de url is vanaf de ogc api process functie naar de ogc api features op dezelfde server is de url zonder poort 5000
input = 'http://localhost/collections/knmi_meetstations/items?f=json&limit=1000' 
result = calc_lof(input)

print(result)
with open('meetstations.geojson', 'w') as f:
    json.dump(json.loads(result['value']), f)

m = folium.Map(location=[52.2, 5.5], zoom_start=8)
gdf = gpd.read_file('meetstations.geojson')
gdf2 = gdf[['STN','TYPE', 'geometry','abnormality']]


folium.GeoJson(
    gdf2,
    name="Stations",
    marker=folium.Marker(icon=folium.Icon(icon='star')),
    tooltip=folium.GeoJsonTooltip(fields=["STN","abnormality"]),
    popup=folium.GeoJsonPopup(fields=["STN", "TYPE"]),
    style_function=lambda x: {
        'markerColor': 'orange' if x['properties']['abnormality'] == -1 else 'green',
        'color': 'black',
        'weight': 2,
        'fillOpacity': 0.7
    },
    highlight_function=lambda x: {"fillOpacity": 0.8},
).add_to(m)

value_counts = gdf['abnormality'].value_counts()
print(value_counts)
m

{'id': 'output_dataset', 'value': '{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"STN": 201, "TYPE": "Platform/AWS", "abnormality": 1}, "geometry": {"type": "Point", "coordinates": [2.9358, 54.3256]}}, {"id": "1", "type": "Feature", "properties": {"STN": 203, "TYPE": "Platform/AWS", "abnormality": 1}, "geometry": {"type": "Point", "coordinates": [3.3417, 52.36]}}, {"id": "2", "type": "Feature", "properties": {"STN": 204, "TYPE": "Platform/AWS", "abnormality": 1}, "geometry": {"type": "Point", "coordinates": [3.6278, 53.2694]}}, {"id": "3", "type": "Feature", "properties": {"STN": 205, "TYPE": "Platform/AWS", "abnormality": 1}, "geometry": {"type": "Point", "coordinates": [3.8103, 55.3992]}}, {"id": "4", "type": "Feature", "properties": {"STN": 206, "TYPE": "Platform/AWS", "abnormality": 1}, "geometry": {"type": "Point", "coordinates": [4.0122, 54.1167]}}, {"id": "5", "type": "Feature", "properties": {"STN": 207, "TYPE": "Platform/AWS", "abnorm

### OGC API Feature request Bomen

In [9]:
m = folium.Map(location=[52.33, 4.8], zoom_start=15)

url = "http://localhost:5000/collections/bomen/items?f=json&limit=1000"

# Fetch the GeoJSON data
gdf = gpd.read_file(url)

#quick hack omdat de datetime kolom niet goed naar geojson geconverteerd wordt...
gdf_subset = gdf[['id','species:nl', 'geometry']]

# Convert to GeoJSON
geojson = gdf_subset.to_json()

folium.GeoJson(
    gdf_subset,
    name="Bomen",
    style_function=lambda x: {'fillColor': 'blue', 'color': 'black', 'weight': 1, 'fillOpacity': 0.5},
    tooltip=folium.GeoJsonTooltip(fields=["id"]),
    popup=folium.GeoJsonPopup(fields=["id", "species:nl"]),
    highlight_function=lambda x: {"fillOpacity": 0.8},
).add_to(m)

m

__Input in de 'aanvraag service'__

- Boom Identificatie = object_id: 2069296
- Betrokkene = subject_id: 'Meneer van Eik'

In [ ]:
def calc_aanvraag(feat):
    url = "http://localhost:5000/processes/aanvraag/execution"  
    data = {"inputs": {"dataset": feat, "object_id": 2069296, "subject_id": 'Meneer van Eik'}}
    headers = {"Content-Type": "application/json"}

    response = requests.post(url, json=data, headers=headers)
    return handle_response(response)


#let op: omdat dit de url is vanaf de ogc api process functie naar de ogc api features op dezelfde server is de url zonder poort 5000
input = 'http://localhost/collections/bomen/items?f=json&limit=1000' 
result = calc_aanvraag(input)

print(result)
with open('bomen.geojson', 'w') as f:
    json.dump(json.loads(result['value']), f)

m = folium.Map(location=[52.33, 4.8], zoom_start=15)
gdf = gpd.read_file('bomen.geojson')
gdf2 = gdf[['id','species:nl', 'kap_aanvraag', 'geometry']]


folium.GeoJson(
    gdf2,
    name="Bomen",
    marker=folium.Marker(icon=folium.Icon(icon='tree')),
    tooltip=folium.GeoJsonTooltip(fields=["id","species:nl"]),
    popup=folium.GeoJsonPopup(fields=["id", "species:nl","kap_aanvraag"]),
    style_function=lambda x: {
        'markerColor': 'orange' if x['properties']['kap_aanvraag'] != '0' else 'green',
        'color': 'black',
        'weight': 2,
        'fillOpacity': 0.7
    },
    highlight_function=lambda x: {"fillOpacity": 0.8},
).add_to(m)

value_counts = gdf['kap_aanvraag'].value_counts()
print(value_counts)
m

{'id': 'output_dataset', 'value': '{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"id": "1173897", "leaf_type": "broadleaved", "species:nl": "Gewone esdoorn", "kap_aanvraag": "0"}, "geometry": {"type": "Point", "coordinates": [4.8028939, 52.336442]}}, {"id": "1", "type": "Feature", "properties": {"id": "1106161", "leaf_type": "broadleaved", "species:nl": "Gewone plataan", "kap_aanvraag": "0"}, "geometry": {"type": "Point", "coordinates": [4.80159, 52.337794]}}, {"id": "2", "type": "Feature", "properties": {"id": "1105975", "leaf_type": "broadleaved", "species:nl": "Zomereik", "kap_aanvraag": "0"}, "geometry": {"type": "Point", "coordinates": [4.806613, 52.337597]}}, {"id": "3", "type": "Feature", "properties": {"id": "1131902", "leaf_type": "broadleaved", "species:nl": "Boomhazelaar", "kap_aanvraag": "0"}, "geometry": {"type": "Point", "coordinates": [4.8138001, 52.337522]}}, {"id": "4", "type": "Feature", "properties": {"id": "1173150", "leaf_